# Training SimpleNN on CIFAR-10
In this project, you will use the SimpleNN model to perform image classification on CIFAR-10. CIFAR-10 orginally contains 60K images from 10 categories. We split it into 45K/5K/10K images to serve as train/valiation/test set. We only release the ground-truth labels of training/validation dataset to you.

## Step 0: Set up the SimpleNN model
As you have practiced to implement simple neural networks in Homework 1, we just prepare the implementation for you.

In [1]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ResBlock(nn.Module):
    def __init__(self,channel,filter_size=3):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(channel, channel, filter_size,padding=1,bias=False)
        nn.init.kaiming_normal_(self.conv1.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.norm1 = nn.BatchNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, channel, filter_size,padding=1,bias=False)
        nn.init.kaiming_normal_(self.conv2.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.norm2 = nn.BatchNorm2d(channel)
        self.norm3 = nn.BatchNorm2d(channel)
    def forward(self,x):
        out=F.leaky_relu(self.norm1(self.conv1(x)))
        out=F.leaky_relu(self.norm2(self.conv2(out)))
        out=self.norm3(out+x)
        return out

class ResDownSampling(nn.Module):
    def __init__(self,channel,out_channel,filter_size=3):
        super(ResDownSampling, self).__init__()
        self.conv1 = nn.Conv2d(channel, channel, filter_size,stride=2,padding=1,bias=False)
        nn.init.kaiming_normal_(self.conv1.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.norm1 = nn.BatchNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, out_channel, filter_size,padding=1,bias=False)
        nn.init.kaiming_normal_(self.conv2.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.norm2 = nn.BatchNorm2d(out_channel)
        self.norm3 = nn.BatchNorm2d(out_channel)
        self.conv3 = nn.Conv2d(channel,out_channel,kernel_size=1,stride=2,bias=False)
        nn.init.kaiming_normal_(self.conv3.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.norm4 = nn.BatchNorm2d(out_channel)
    def forward(self,x):
        out=F.leaky_relu(self.norm1(self.conv1(x)))
        out=F.leaky_relu(self.norm2(self.conv2(out)))
        x=F.leaky_relu(self.norm3(self.conv3(x)))
        out=self.norm4(out+x)
        return out
class ResNetCIFAR(nn.Module):
    def __init__(self ):
        super(ResNetCIFAR, self).__init__()
        self.inconv=nn.Conv2d(3, 16,3,padding=1,bias=False)
        nn.init.xavier_normal_(self.inconv.weight)
        self.res_block=nn.Sequential(self.inconv,
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(),
                                    ResBlock(16),
                                    ResBlock(16),
                                    ResBlock(16),
                                    ResDownSampling(16,32),
                                    ResBlock(32),
                                    ResBlock(32),
                                    ResDownSampling(32,64),
                                    ResBlock(64),
                                    ResBlock(64),
                                    nn.AvgPool2d(8),
                                    nn.Flatten(),
                                    nn.Linear(64*1*1,10))
    def forward(self, x):
        return self.res_block(x)


### Question (a)
Here is a sanity check to verify the implementation of SimpleNN. 
You need to:
1. Write down your code.
2. **In the PDF report**, give a brief description on how the code helps you know that SimpleNN is implemented correctly.

In [3]:
#############################################
# your code here
# sanity check for the correctness of SimpleNN
model=ResNetCIFAR()
model.eval()
test_tensor=torch.zeros(2,3,32,32)
out=model(test_tensor)
assert(out.shape==torch.Size([2,10]))
assert(torch.sum(out[0]-out[1]).item()==0)


#############################################

## Step 1: Set up preprocessing functions
Preprocessing is very important as discussed in the lecture.
You will need to write preprocessing functions with the help of *torchvision.transforms* in this step.
You can find helpful tutorial/API at [here](https://pytorch.org/vision/stable/transforms.html).

### Question (b)
For the question, you need to:
1. Complete the preprocessing code below.
2. **In the PDF report**, briefly describe what preprocessing operations you used and what are the purposes of them.

Hint: 
1. Only two operations are necessary to complete the basic preprocessing here.
2. The raw input read from the dataset will be PIL images.
3. Data augmentation operations are not mendatory, but feel free to incorporate them if you want.
4. Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [4]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
transform = transforms.Compose(
    (
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    )
)
transform_train = transforms.Compose(
    (
    
    transforms.RandomCrop((32,32),padding=4),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    
    #
    #transforms.ColorJitter(0.2,0,0)
    
    )
)

transform_val = transform
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [5]:
# do NOT change these
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, 
    train=True, 
    download=True,
    transform=transform_train    # your code
)

val_set = CIFAR10(
    root=DATA_ROOT, 
    train=False, 
    download=True,
    transform=transform_val    # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=TRAIN_BATCH_SIZE,  # your code
    shuffle=True,     # your code
    num_workers=2
)

val_loader = DataLoader(
    val_set, 
    batch_size=VAL_BATCH_SIZE,  # your code
    shuffle=False,     # your code
    num_workers=2
)
#############################################

Files already downloaded and verified
Files already downloaded and verified


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

In [6]:
# specify the device for computation
#############################################
# your code here
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model=ResNetCIFAR()
model=model.to(device)
#############################################

## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum. 

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [7]:
import torch.nn as nn
import torch.optim as optim

# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.10114514

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 5e-4

#############################################
# your code here
# create loss function
criterion = nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(model.parameters(),momentum=MOMENTUM, nesterov=True ,lr=INITIAL_LR,weight_decay=REG)
#############################################

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps: 

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


In [8]:
# some hyperparameters
# total number of training epochs
EPOCHS = 200

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"
DECAY_EPOCHS=70
DECAY=0.1
# start the training/validation process
# the process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
best_val_acc = 0
current_learning_rate = INITIAL_LR

print("==> Training starts!")
print("="*50)
for i in range(0, EPOCHS):
    # handle the learning rate scheduler.
    
    if i % DECAY_EPOCHS == 0 and i != 0 :
        current_learning_rate = current_learning_rate * DECAY
    
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
    
    #######################
    # your code here
    # switch to train mode
    model.train()
    
    #######################
    
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want
    loader=train_loader
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(loader):
        ####################################
        # your code here
        # copy inputs to device
        inputs=inputs.to(device)
        targets=targets.to(device).long()

        
        # compute the output and loss
        out=model(inputs)
        loss=criterion(out,targets)
        
        # zero the gradient
        
        optimizer.zero_grad()
        # backpropagation
        loss.backward()

        
        # apply gradient and update the weights
        optimizer.step()
        train_loss+=loss.item()
        
        # count the number of correctly predicted samples in the current batch
        correct_examples+=torch.sum(out.argmax(-1)==targets).item()
        ####################################
    total_examples=len(train_loader.dataset)      
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # Validate on the validation dataset
    #######################
    # your code here
    # switch to eval mode
    model.eval()
    
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs=inputs.to(device)
            targets=targets.to(device).long()
            # compute the output and loss
            out=model(inputs)
            loss=criterion(out,targets)
            # count the number of correctly predicted samples in the current batch
            val_loss+=loss.item()
            correct_examples+=torch.sum(out.argmax(-1)==targets).item()
            
            ####################################
    total_examples=len(val_loader.dataset)
    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    
    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
            os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': model.state_dict(),
                 'epoch': i,
                 'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

==> Training starts!
Epoch 0:
Training loss: 1.4280, Training accuracy: 0.4748
Validation loss: 1.4127, Validation accuracy: 0.5152
Saving ...

Epoch 1:
Training loss: 0.9786, Training accuracy: 0.6530
Validation loss: 0.9241, Validation accuracy: 0.6768
Saving ...

Epoch 2:
Training loss: 0.8036, Training accuracy: 0.7184
Validation loss: 0.8576, Validation accuracy: 0.7002
Saving ...

Epoch 3:
Training loss: 0.7083, Training accuracy: 0.7532
Validation loss: 0.8543, Validation accuracy: 0.7137
Saving ...

Epoch 4:
Training loss: 0.6599, Training accuracy: 0.7715
Validation loss: 0.9048, Validation accuracy: 0.7029

Epoch 5:
Training loss: 0.6281, Training accuracy: 0.7819
Validation loss: 0.6251, Validation accuracy: 0.7877
Saving ...

Epoch 6:
Training loss: 0.6000, Training accuracy: 0.7938
Validation loss: 0.7732, Validation accuracy: 0.7482

Epoch 7:
Training loss: 0.5768, Training accuracy: 0.8007
Validation loss: 0.6800, Validation accuracy: 0.7754

Epoch 8:
Training loss: 0.56

# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```